In [ ]:
# import dependencies
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


In [ ]:
START = datetime(2016, 1, 1)
END = datetime(2021, 2, 16)
BUCKET = 1  # day

COLUMNS = ["VOLUME_fut", "VOLUME_etf", "VOLUME_opt"] # change
TARGET = "VOLUME_fut_target" # change

In [ ]:
# Load data
global_markets_volumes_path = "./data/VOLUMES.csv"
vix_path = "./data/VIX.csv"
sp500_path = "./data/SP500.csv"
gold_path= "./data/GOLD.csv"
euro_usd_path = "./data/EURUSD.csv"
bonds_path = "./data/10YBONDYIELDS.csv"

In [ ]:
def load_data(path: str, target:str) -> pd.DataFrame:
    df = pd.read_csv('./data/etf_nq.csv', index_col='Date')
    # rename columns to lower case
    df.columns = map(str.lower, df.columns)
    new_columns = [column + f"_{target}" for column in df.columns if column != 'date']
    # Add _target to columns except Date
    df.rename(columns=dict(zip(df.columns[1:], new_columns)),inplace=True)
    # Remove space from columns
    df.columns = df.columns.str.replace(" ", "_")
    return df